In [1]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 3

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Using TensorFlow backend.
Saving vectors of label - 'cat': 100%|██████████| 1733/1733 [00:19<00:00, 89.63it/s] 


In [2]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [3]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3112 samples, validate on 2076 samples
Epoch 1/50
3112/3112 [==============================] - 5s 2ms/step - loss: 1.6072 - acc: 0.4566 - val_loss: 0.8331 - val_acc: 0.5891
Epoch 2/50
3112/3112 [==============================] - 2s 623us/step - loss: 0.7199 - acc: 0.6880 - val_loss: 0.5555 - val_acc: 0.7905
Epoch 3/50
3112/3112 [==============================] - 2s 635us/step - loss: 0.5228 - acc: 0.8004 - val_loss: 0.3931 - val_acc: 0.8593
Epoch 4/50
3112/3112 [==============================] - 2s 655us/step - loss: 0.4391 - acc: 0.8470 - val_loss: 0.4156 - val_acc: 0.8545
Epoch 5/50
3112/3112 [==============================] - 2s 644us/step - loss: 0.3468 - acc: 0.8756 - val_loss: 0.3119 - val_acc: 0.8820
Epoch 6/50
3112/3112 [==============================] - 2s 641us/step - loss: 0.2936 - acc: 0.8952 - val_loss: 0.2480 - val_acc: 0.9090
Epoch 7/50
3112/3112 [==============================] - 2s 640us/step - loss: 0.2423 - acc: 0.9161 - val_loss: 0.2745 - val_acc: 0.8960
Ep

## Prediction

In [7]:
print(predict('./0fa1e7a9_nohash_0.wav', model=model))

cat
